In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd 
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
import GPy
import matplotlib.pyplot as plt 
%matplotlib inline

from dispersant_screener.gp import build_model, build_coregionalized_model, predict_coregionalized, set_xy_coregionalized
from dispersant_screener.pal import pal
from dispersant_screener.utils import get_kmeans_samples

import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
other_descriptors = ['CellV [A^3]']

geometric_descirptors = [
 'Di',
 'Df',
 'Dif',
 'density [g/cm^3]',
 'total_SA_volumetric',
 'total_SA_gravimetric',
 'total_POV_volumetric',
 'total_POV_gravimetric'
 ]

linker_descriptors = [
 'f-lig-chi-0',
 'f-lig-chi-1',
 'f-lig-chi-2',
 'f-lig-chi-3',
 'f-lig-Z-0',
 'f-lig-Z-1',
 'f-lig-Z-2',
 'f-lig-Z-3',
 'f-lig-I-0',
 'f-lig-I-1',
 'f-lig-I-2',
 'f-lig-I-3',
 'f-lig-T-0',
 'f-lig-T-1',
 'f-lig-T-2',
 'f-lig-T-3',
 'f-lig-S-0',
 'f-lig-S-1',
 'f-lig-S-2',
 'f-lig-S-3',
 'lc-chi-0-all',
 'lc-chi-1-all',
 'lc-chi-2-all',
 'lc-chi-3-all',
 'lc-Z-0-all',
 'lc-Z-1-all',
 'lc-Z-2-all',
 'lc-Z-3-all',
 'lc-I-0-all',
 'lc-I-1-all',
 'lc-I-2-all',
 'lc-I-3-all',
 'lc-T-0-all',
 'lc-T-1-all',
 'lc-T-2-all',
 'lc-T-3-all',
 'lc-S-0-all',
 'lc-S-1-all',
 'lc-S-2-all',
 'lc-S-3-all',
 'lc-alpha-0-all',
 'lc-alpha-1-all',
 'lc-alpha-2-all',
 'lc-alpha-3-all',
 'D_lc-chi-0-all',
 'D_lc-chi-1-all',
 'D_lc-chi-2-all',
 'D_lc-chi-3-all',
 'D_lc-Z-0-all',
 'D_lc-Z-1-all',
 'D_lc-Z-2-all',
 'D_lc-Z-3-all',
 'D_lc-I-0-all',
 'D_lc-I-1-all',
 'D_lc-I-2-all',
 'D_lc-I-3-all',
 'D_lc-T-0-all',
 'D_lc-T-1-all',
 'D_lc-T-2-all',
 'D_lc-T-3-all',
 'D_lc-S-0-all',
 'D_lc-S-1-all',
 'D_lc-S-2-all',
 'D_lc-S-3-all',
 'D_lc-alpha-0-all',
 'D_lc-alpha-1-all',
 'D_lc-alpha-2-all',
 'D_lc-alpha-3-all']

metalcenter_descriptors =[
 'mc_CRY-chi-0-all',
 'mc_CRY-chi-1-all',
 'mc_CRY-chi-2-all',
 'mc_CRY-chi-3-all',
 'mc_CRY-Z-0-all',
 'mc_CRY-Z-1-all',
 'mc_CRY-Z-2-all',
 'mc_CRY-Z-3-all',
 'mc_CRY-I-0-all',
 'mc_CRY-I-1-all',
 'mc_CRY-I-2-all',
 'mc_CRY-I-3-all',
 'mc_CRY-T-0-all',
 'mc_CRY-T-1-all',
 'mc_CRY-T-2-all',
 'mc_CRY-T-3-all',
 'mc_CRY-S-0-all',
 'mc_CRY-S-1-all',
 'mc_CRY-S-2-all',
 'mc_CRY-S-3-all',
 'D_mc_CRY-chi-0-all',
 'D_mc_CRY-chi-1-all',
 'D_mc_CRY-chi-2-all',
 'D_mc_CRY-chi-3-all',
 'D_mc_CRY-Z-0-all',
 'D_mc_CRY-Z-1-all',
 'D_mc_CRY-Z-2-all',
 'D_mc_CRY-Z-3-all',
 'D_mc_CRY-I-0-all',
 'D_mc_CRY-I-1-all',
 'D_mc_CRY-I-2-all',
 'D_mc_CRY-I-3-all',
 'D_mc_CRY-T-0-all',
 'D_mc_CRY-T-1-all',
 'D_mc_CRY-T-2-all',
 'D_mc_CRY-T-3-all',
 'D_mc_CRY-S-0-all',
 'D_mc_CRY-S-1-all',
 'D_mc_CRY-S-2-all',
 'D_mc_CRY-S-3-all']

functionalgroup_descriptors = [
 'func-chi-0-all',
 'func-chi-1-all',
 'func-chi-2-all',
 'func-chi-3-all',
 'func-Z-0-all',
 'func-Z-1-all',
 'func-Z-2-all',
 'func-Z-3-all',
 'func-I-0-all',
 'func-I-1-all',
 'func-I-2-all',
 'func-I-3-all',
 'func-T-0-all',
 'func-T-1-all',
 'func-T-2-all',
 'func-T-3-all',
 'func-S-0-all',
 'func-S-1-all',
 'func-S-2-all',
 'func-S-3-all',
 'func-alpha-0-all',
 'func-alpha-1-all',
 'func-alpha-2-all',
 'func-alpha-3-all',
 'D_func-chi-0-all',
 'D_func-chi-1-all',
 'D_func-chi-2-all',
 'D_func-chi-3-all',
 'D_func-Z-0-all',
 'D_func-Z-1-all',
 'D_func-Z-2-all',
 'D_func-Z-3-all',
 'D_func-I-0-all',
 'D_func-I-1-all',
 'D_func-I-2-all',
 'D_func-I-3-all',
 'D_func-T-0-all',
 'D_func-T-1-all',
 'D_func-T-2-all',
 'D_func-T-3-all',
 'D_func-S-0-all',
 'D_func-S-1-all',
 'D_func-S-2-all',
 'D_func-S-3-all',
 'D_func-alpha-0-all',
 'D_func-alpha-1-all',
 'D_func-alpha-2-all',
 'D_func-alpha-3-all']


summed_linker_descriptors = [
 'sum-f-lig-chi-0',
 'sum-f-lig-chi-1',
 'sum-f-lig-chi-2',
 'sum-f-lig-chi-3',
 'sum-f-lig-Z-0',
 'sum-f-lig-Z-1',
 'sum-f-lig-Z-2',
 'sum-f-lig-Z-3',
 'sum-f-lig-I-0',
 'sum-f-lig-I-1',
 'sum-f-lig-I-2',
 'sum-f-lig-I-3',
 'sum-f-lig-T-0',
 'sum-f-lig-T-1',
 'sum-f-lig-T-2',
 'sum-f-lig-T-3',
 'sum-f-lig-S-0',
 'sum-f-lig-S-1',
 'sum-f-lig-S-2',
 'sum-f-lig-S-3',
 'sum-lc-chi-0-all',
 'sum-lc-chi-1-all',
 'sum-lc-chi-2-all',
 'sum-lc-chi-3-all',
 'sum-lc-Z-0-all',
 'sum-lc-Z-1-all',
 'sum-lc-Z-2-all',
 'sum-lc-Z-3-all',
 'sum-lc-I-0-all',
 'sum-lc-I-1-all',
 'sum-lc-I-2-all',
 'sum-lc-I-3-all',
 'sum-lc-T-0-all',
 'sum-lc-T-1-all',
 'sum-lc-T-2-all',
 'sum-lc-T-3-all',
 'sum-lc-S-0-all',
 'sum-lc-S-1-all',
 'sum-lc-S-2-all',
 'sum-lc-S-3-all',
 'sum-lc-alpha-0-all',
 'sum-lc-alpha-1-all',
 'sum-lc-alpha-2-all',
 'sum-lc-alpha-3-all',
 'sum-D_lc-chi-0-all',
 'sum-D_lc-chi-1-all',
 'sum-D_lc-chi-2-all',
 'sum-D_lc-chi-3-all',
 'sum-D_lc-Z-0-all',
 'sum-D_lc-Z-1-all',
 'sum-D_lc-Z-2-all',
 'sum-D_lc-Z-3-all',
 'sum-D_lc-I-0-all',
 'sum-D_lc-I-1-all',
 'sum-D_lc-I-2-all',
 'sum-D_lc-I-3-all',
 'sum-D_lc-T-0-all',
 'sum-D_lc-T-1-all',
 'sum-D_lc-T-2-all',
 'sum-D_lc-T-3-all',
 'sum-D_lc-S-0-all',
 'sum-D_lc-S-1-all',
 'sum-D_lc-S-2-all',
 'sum-D_lc-S-3-all',
 'sum-D_lc-alpha-0-all',
 'sum-D_lc-alpha-1-all',
 'sum-D_lc-alpha-2-all',
 'sum-D_lc-alpha-3-all']

summed_metalcenter_descriptors =[
 'sum-mc_CRY-chi-0-all',
 'sum-mc_CRY-chi-1-all',
 'sum-mc_CRY-chi-2-all',
 'sum-mc_CRY-chi-3-all',
 'sum-mc_CRY-Z-0-all',
 'sum-mc_CRY-Z-1-all',
 'sum-mc_CRY-Z-2-all',
 'sum-mc_CRY-Z-3-all',
 'sum-mc_CRY-I-0-all',
 'sum-mc_CRY-I-1-all',
 'sum-mc_CRY-I-2-all',
 'sum-mc_CRY-I-3-all',
 'sum-mc_CRY-T-0-all',
 'sum-mc_CRY-T-1-all',
 'sum-mc_CRY-T-2-all',
 'sum-mc_CRY-T-3-all',
 'sum-mc_CRY-S-0-all',
 'sum-mc_CRY-S-1-all',
 'sum-mc_CRY-S-2-all',
 'sum-mc_CRY-S-3-all',
 'sum-D_mc_CRY-chi-0-all',
 'sum-D_mc_CRY-chi-1-all',
 'sum-D_mc_CRY-chi-2-all',
 'sum-D_mc_CRY-chi-3-all',
 'sum-D_mc_CRY-Z-0-all',
 'sum-D_mc_CRY-Z-1-all',
 'sum-D_mc_CRY-Z-2-all',
 'sum-D_mc_CRY-Z-3-all',
 'sum-D_mc_CRY-I-0-all',
 'sum-D_mc_CRY-I-1-all',
 'sum-D_mc_CRY-I-2-all',
 'sum-D_mc_CRY-I-3-all',
 'sum-D_mc_CRY-T-0-all',
 'sum-D_mc_CRY-T-1-all',
 'sum-D_mc_CRY-T-2-all',
 'sum-D_mc_CRY-T-3-all',
 'sum-D_mc_CRY-S-0-all',
 'sum-D_mc_CRY-S-1-all',
 'sum-D_mc_CRY-S-2-all',
 'sum-D_mc_CRY-S-3-all']

summed_functionalgroup_descriptors = [
 'sum-func-chi-0-all',
 'sum-func-chi-1-all',
 'sum-func-chi-2-all',
 'sum-func-chi-3-all',
 'sum-func-Z-0-all',
 'sum-func-Z-1-all',
 'sum-func-Z-2-all',
 'sum-func-Z-3-all',
 'sum-func-I-0-all',
 'sum-func-I-1-all',
 'sum-func-I-2-all',
 'sum-func-I-3-all',
 'sum-func-T-0-all',
 'sum-func-T-1-all',
 'sum-func-T-2-all',
 'sum-func-T-3-all',
 'sum-func-S-0-all',
 'sum-func-S-1-all',
 'sum-func-S-2-all',
 'sum-func-S-3-all',
 'sum-func-alpha-0-all',
 'sum-func-alpha-1-all',
 'sum-func-alpha-2-all',
 'sum-func-alpha-3-all',
 'sum-D_func-chi-0-all',
 'sum-D_func-chi-1-all',
 'sum-D_func-chi-2-all',
 'sum-D_func-chi-3-all',
 'sum-D_func-Z-0-all',
 'sum-D_func-Z-1-all',
 'sum-D_func-Z-2-all',
 'sum-D_func-Z-3-all',
 'sum-D_func-I-0-all',
 'sum-D_func-I-1-all',
 'sum-D_func-I-2-all',
 'sum-D_func-I-3-all',
 'sum-D_func-T-0-all',
 'sum-D_func-T-1-all',
 'sum-D_func-T-2-all',
 'sum-D_func-T-3-all',
 'sum-D_func-S-0-all',
 'sum-D_func-S-1-all',
 'sum-D_func-S-2-all',
 'sum-D_func-S-3-all',
 'sum-D_func-alpha-0-all',
 'sum-D_func-alpha-1-all',
 'sum-D_func-alpha-2-all',
 'sum-D_func-alpha-3-all']


In [5]:
pmof = pd.read_csv('/Users/kevinmaikjablonka/Dropbox (LSMO)/proj70_diversity_mohamad/PMOF20K_traindata_7000_test.csv')

In [6]:
pmof['CO2_DC'] = pmof['pure_uptake_CO2_298.00_1600000'] - pmof['pure_uptake_CO2_298.00_15000']

In [7]:
y = pmof[['CO2_DC', 'CH4DC']].values

In [8]:
feat = set(
["func-chi-0-all" ,"D_func-S-3-all", "total_SA_volumetric", 
 "Di", "Dif", "mc_CRY-Z-0-all","total_POV_volumetric","density [g/cm^3]", "total_SA_gravimetric",
 "D_func-S-1-all","Df", "mc_CRY-S-0-all" ,"total_POV_gravimetric","D_func-alpha-1-all","func-S-0-all",
 "D_mc_CRY-chi-3-all","D_mc_CRY-chi-1-all","func-alpha-0-all",
 "D_mc_CRY-T-2-all","mc_CRY-Z-2-all","D_mc_CRY-chi-2-all",
"total_SA_gravimetric","total_POV_gravimetric","Di","density [g/cm^3]",
 "func-S-0-all",
 "func-chi-2-all","func-alpha-0-all",
 "total_POV_volumetric","D_func-alpha-1-all","total_SA_volumetric",
 "func-alpha-1-all",
 "func-alpha-3-all",
 "Dif",
 "Df",
 "func-chi-3-all"
])

In [9]:
X = pmof[feat]

In [11]:
X = StandardScaler().fit_transform(X)
X = VarianceThreshold(0.2).fit_transform(X)

In [42]:
X_train, y_train, indices = get_kmeans_samples(X, y, 101)

In [39]:
model = build_coregionalized_model(X_train, y_train)

In [35]:
%%time
model.optimize()

CPU times: user 53.6 s, sys: 4.79 s, total: 58.4 s
Wall time: 37.3 s


In [40]:
model

gp.,value,constraints,priors
ICM.RatQuad.variance,1.0,+ve,
ICM.RatQuad.lengthscale,"(25,)",+ve,
ICM.RatQuad.power,2.0,+ve,
ICM.B.W,"(2, 1)",,
ICM.B.kappa,"(2,)",+ve,
mixed_noise.Gaussian_noise_0.variance,1.0,+ve,
mixed_noise.Gaussian_noise_1.variance,1.0,+ve,


In [43]:
m_ = set_xy_coregionalized(model, X_train, y_train)

In [47]:
predict_coregionalized(m_, X_train)

(array([[ 3.49032298],
        [ 5.0848422 ],
        [ 0.82499055],
        [11.24510216],
        [ 1.39274898],
        [14.67118695],
        [ 2.22116637],
        [ 9.5780421 ],
        [17.26989911],
        [ 5.63843934],
        [ 3.82318218],
        [ 4.84283578],
        [ 3.98553742],
        [ 4.78025852],
        [ 6.65872084],
        [ 6.40090176],
        [ 1.33840219],
        [ 2.80543182],
        [ 8.53389845],
        [16.70520706],
        [13.63262592],
        [12.40933502],
        [15.92096675],
        [11.52114572],
        [ 1.78993423],
        [11.85836731],
        [ 6.0156004 ],
        [ 5.63381285],
        [ 6.45080578],
        [ 2.96095793],
        [10.82835563],
        [ 2.05955178],
        [10.87410472],
        [ 4.49837742],
        [ 1.99813091],
        [ 0.88259091],
        [12.20318107],
        [ 4.47983006],
        [11.5511782 ],
        [15.41610034],
        [ 4.15498115],
        [ 3.46986269],
        [ 0.64444288],
        [ 2

In [ ]:
GPy.Co